[View in Colaboratory](https://colab.research.google.com/github/hdh7485/cnn-text-classification-tf/blob/master/NavScript.ipynb)

##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [71]:
!ps ax|grep python

   89 ?        Sl     0:03 /usr/bin/python2 /usr/local/bin/jupyter-notebook -y --no-browser --log-level=DEBUG --debug --NotebookApp.allow_origin="*" --NotebookApp.log_format="%(message)s" --NotebookApp.token= --Session.key="" --Session.keyfile="" --ContentsManager.untitled_directory="Untitled Folder" --ContentsManager.untitled_file="Untitled File" --ContentsManager.untitled_notebook="Untitled Notebook" --NotebookNotary.algorithm="sha1" --KernelManager.autorestart=True --MultiKernelManager.default_kernel_name="python2" --ip="127.0.0.1" --port=9000 --port-retries=0 --notebook-dir="/content" --NotebookNotary.algorithm=sha256 --NotebookNotary.secret_file=/content/datalab/.config/notary_secret --NotebookApp.base_url=/tun/m/gpu-cc67aecd-ee17-4bb8-8158-8ded892e6622/
   97 ?        Ssl    5:26 /usr/bin/python3 -m ipykernel_launcher -f /content/.local/share/jupyter/runtime/kernel-b880ed7c-d88e-415c-a2f8-571629d14dce.json
 9223 pts/0    Ss+    0:01 /bin/sh -c ps ax|grep python
 9225 pts/0    S+ 

In [72]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

In [73]:
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 8.8 GB  I Proc size: 4.3 GB
GPU RAM Free: 548MB | Used: 10891MB | Util  95% | Total 11439MB


# Universal Sentence Encoder


<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>


This notebook illustrates how to access the Universal Sentence Encoder and use it for sentence similarity and sentence classification tasks.

The Universal Sentence Encoder makes getting sentence level embeddings as easy as it has historically been to lookup the embeddings for individual words. The sentence embeddings can then be trivially used to compute sentence level meaning similarity as well as to enable better performance on downstream classification tasks using less supervised training data.


# Getting Started

This section sets up the environment for access to the Universal Sentence Encoder on TF Hub and provides examples of applying the encoder to words, sentences, and paragraphs.

In [74]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
!pip3 install seaborn

More detailed information about installing Tensorflow can be found at [https://www.tensorflow.org/install/](https://www.tensorflow.org/install/).

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [0]:
def rmse(predictions, targets):
  return np.sqrt(((predictions - targets)**2).mean())

In [0]:
# 미리 학습된 모듈의 url
module_url = "https://tfhub.dev/google/universal-sentence-encoder/1" #@param ["https://tfhub.dev/google/universal-sentence-encoder/1", "https://tfhub.dev/google/universal-sentence-encoder-large/1"]

In [0]:
# Compute a representation for each message, showing various lengths supported.
messages = ["What's the {OTHER:COMMON} for this afternoon?",
            "What's the {OTHER:COMMON} like on my {OTHER:COMMON}?",
            "Show me a {OTHER:COMMON} on {LOCATION:PROPER} and {LOCATION:PROPER}.",
            "Can you find me a {LOCATION:COMMON} with {LOCATION:COMMON} nearby?",
            "Find a {LOCATION:COMMON} along {LOCATION:COMMON}.",
            "Find the cheapest indoor {OTHER:COMMON} within 500 meters of my {OTHER:COMMON}.",
            "Okay, can you find me a {LOCATION:COMMON} on my {OTHER:COMMON} that has a {LOCATION:COMMON}?",
            "Find {OTHER:COMMON} near {LOCATION:COMMON} that accepts {OTHER:COMMON} and has a {OTHER:COMMON}.",
            
            "Navigate to {LOCATION:PROPER}.",
            "What's my {OTHER:PROPER} to {LOCATION:COMMON}?",
            "Show me alternative {OTHER:COMMON}.",
            "Reroute using {OTHER:PROPER}.",
            
            
            "Drive to {LOCATION:PROPER}.",
            "What's my {OTHER:COMMON}?",
            "Can I make tomorrow's 10am {EVENT:COMMON} without recharging?",
            "What's {OTHER:COMMON} like on the {LOCATION:PROPER}?",
            "Are there any {OTHER:COMMON} on my {OTHER:COMMON}?",
            "Will it rain tomorrow in {LOCATION:PROPER}?"
            ]

# Compute a representation for each message, showing various lengths supported.
messages2 = ["What's the weather forecast for this afternoon?",
            "What's the traffic like on my route?",
            "Show me a Traffic Camera on US-101 and Bayshore Blvd.",
            "Can you find me a gas station with restroom facilities nearby?",
            "Find a coffee shop along route",
            "Find the cheapest indoor parking within 500 meters of my destination.",
            "Okay, can you find me a supermarket on my route that has a charging station?",
            "Find parking near destination that accepts credit cards and has a valet service.",
            "Navigate to San Francisco Museum of Modern Art.",
            "What's my ETA to destination?",
            "Show me alternative routes.",
            "Reroute using I-580 East.",
  
            "Drive to Downtown Berkeley.",
            "What's my drive range?",
            "Can I make tomorrow's 10am meeting without recharging?",
            "What's traffic like on the Bay Bridge?",
            "Are there any speed cameras on my route?",
            "Will it rain tomorrow in Oakland?"
            ]

scripts = ["[SEARCH FROM:WEATHERFORECAST WHERE:HERE WHEN:AFTERNOON]",
         "[SEARCH FROM:TRAFFIC WHERE:ONROUTE]",
         "[SEARCH FROM:TRAFFICCAMERA WHERE:[SEARCH GEOCODE WHERE:”US-101 and Bayshore Blvd”]]",
         "[SEARCH FROM:GASSTATION WHERE:NEARBY WITH:RESTROOM]",
         "[SEARCH ONE FROM:COFFEESHOP WHERE:ALONGROUTE]",
         "[SEARCH ONE FROM:OFFROADPARKING WHERE:DESTINATION RANGE:500M WITH:[SORT PRICE ASC]]",
         "[SEARCH ONE FROM:SUPERMARKET WHERE:ONROUTE WITH:CHARGINGSTATION]",
         "[SEARCH ONE FROM:PARKING WITH:CREDITCARD WITH:VALETSERVICE]",
         "[ROUTE TO:[SEARCH KEYWORD:”San Francisco Museum of Modern Art”]]",
         "[ROUTE INFO:ETA]",
         "[ROUTE ALTROUTE]",
         "[ROUTE ALTROUTE USE:[SEARCH LINKS:ROUTE]]",
         
         "[MODE GUIDANCE WITH:[ROUTE TO:[SEARCH KEYWORD:”Downtown Berkeley”]]]",
         "[MODE DRIVERANGE]",
         "[MODE DRIVERANGE TO:[SEARCH KEYWORD:”10AM MEETING” FROM:SCHEDULE WHEN:10AM] WITH:[VOICERESPONSE TEMPLATE:YES/NO*]]",
         "[MODE TRAFFIC [SEARCH FROM:TRAFFIC WHERE:[SEARCH KEYWORD:”Bay Bridge”]] WITH:[VOICERESPONSE TEMPLATE:””*]",
         "[MODE SPEEDCAMERA WHERE:ONROUTE WITH:[VOICERESPONSE TEMPLATE:””*]]",
         "[MODE WEATHERFORECAST WHERE:[SEARCH KEYWORD:”Oakland”] WHEN:TOMORROW]"
          ]

In [0]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

#품사 태깅 문장 임베딩
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))
  
#일반 문장 임베딩
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings2 = session.run(embed(messages2))

In [80]:
for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: What's the {OTHER:COMMON} for this afternoon?
Embedding size: 512
Embedding: [-0.021156975999474525, -0.004225220996886492, -0.016338560730218887, ...]

Message: What's the {OTHER:COMMON} like on my {OTHER:COMMON}?
Embedding size: 512
Embedding: [-0.012699112296104431, 0.0099342567846179, -0.03980620950460434, ...]

Message: Show me a {OTHER:COMMON} on {LOCATION:PROPER} and {LOCATION:PROPER}.
Embedding size: 512
Embedding: [0.004817151930183172, -0.06830794364213943, -0.010058710351586342, ...]

Message: Can you find me a {LOCATION:COMMON} with {LOCATION:COMMON} nearby?
Embedding size: 512
Embedding: [-0.008437147364020348, 0.001783942454494536, -0.007823196239769459, ...]

Message: Find a {LOCATION:COMMON} along {LOCATION:COMMON}.
Embedding size: 512
Embedding: [-0.02695729397237301, -0.03852830082178116, 0.009048445150256157, ...]

Message: Find the cheapest indoor {OTHER:COMMON} within 500 meters of my {OTHER:COMMON}.
Embedding size: 512
Embedding: [0.034149419516325, 0.049

In [0]:
# Compute a representation for each message, showing various lengths supported.
test_message = ["Where is a {OTHER:COMMON} with a {OTHER:COMMON} nearby?",
               "Find routes to {LOCATION:PROPER}.",
               "Find routes to {OTHER:COMMON}.",
               "How is {OTHER:COMMON} in {LOCATION:PROPER} now?",
               "Find a {LOCATION:COMMON} near the {LOCATION:COMMON} on my {OTHER:COMMON}."]

test_message2 = ["Where is a gas station with a restroom facilities nearby?",
               "Find routes to San Francisco Museum of Modern Art.",
               "Find routes to gas station.",
               "How is traffic in Bay Bridge now?",
               "Find a supermarket near the charging station on my route."]

test_labels = [3, 8, 8, 15, 6]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

#테스트용 품사 태깅 문장 임베딩
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  test_message_embeddings = session.run(embed(test_message))
  
#테스트용 일반 문장 임베딩
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  test_message_embeddings2 = session.run(embed(test_message2))

In [85]:
# Word class with symbols
print("Word class with symbols")
for test_message_embedding, test_label in zip(test_message_embeddings, test_labels):
  minimum = 100
  minimum_index = 0
  for i, message_embedding in enumerate(message_embeddings):
    error = rmse(np.array(message_embedding), np.array(test_message_embedding))
    if minimum > error:
      minimum = error
      minimum_index = i

  print("Minimum RMSE value: {}".format(minimum))
  print("Most similar script: {}".format(scripts[minimum_index]))
  print("Estimation: {}".format(minimum_index))
  print("Answer: {}\n".format(test_label))


Word class with symbols
Minimum RMSE value: 0.03831955045461655
Most similar script: [SEARCH ONE FROM:PARKING WITH:CREDITCARD WITH:VALETSERVICE]
Estimation: 7
Answer: 3

Minimum RMSE value: 0.03203499689698219
Most similar script: [ROUTE TO:[SEARCH KEYWORD:”San Francisco Museum of Modern Art”]]
Estimation: 8
Answer: 8

Minimum RMSE value: 0.036623530089855194
Most similar script: [ROUTE TO:[SEARCH KEYWORD:”San Francisco Museum of Modern Art”]]
Estimation: 8
Answer: 8

Minimum RMSE value: 0.041374027729034424
Most similar script: [MODE TRAFFIC [SEARCH FROM:TRAFFIC WHERE:[SEARCH KEYWORD:”Bay Bridge”]] WITH:[VOICERESPONSE TEMPLATE:””*]
Estimation: 15
Answer: 15

Minimum RMSE value: 0.030955029651522636
Most similar script: [SEARCH ONE FROM:COFFEESHOP WHERE:ALONGROUTE]
Estimation: 4
Answer: 6



In [86]:
# Common nouns
print("Common nouns")
for test_message_embedding, test_label in zip(test_message_embeddings2, test_labels):
  minimum = 100
  minimum_index = 0
  for i, message_embedding in enumerate(message_embeddings2):
    error = rmse(np.array(message_embedding), np.array(test_message_embedding))
    if minimum > error:
      minimum = error
      minimum_index = i

  print("Minimum RMSE value: {}".format(minimum))
  print("Most similar script: {}".format(scripts[minimum_index]))
  print("Estimation: {}".format(minimum_index))
  print("Answer: {}\n".format(test_label))


Common nouns
Minimum RMSE value: 0.02820552885532379
Most similar script: [SEARCH FROM:GASSTATION WHERE:NEARBY WITH:RESTROOM]
Estimation: 3
Answer: 3

Minimum RMSE value: 0.020899992436170578
Most similar script: [ROUTE TO:[SEARCH KEYWORD:”San Francisco Museum of Modern Art”]]
Estimation: 8
Answer: 8

Minimum RMSE value: 0.03728972375392914
Most similar script: [SEARCH ONE FROM:SUPERMARKET WHERE:ONROUTE WITH:CHARGINGSTATION]
Estimation: 6
Answer: 8

Minimum RMSE value: 0.022458434104919434
Most similar script: [MODE TRAFFIC [SEARCH FROM:TRAFFIC WHERE:[SEARCH KEYWORD:”Bay Bridge”]] WITH:[VOICERESPONSE TEMPLATE:””*]
Estimation: 15
Answer: 15

Minimum RMSE value: 0.026485448703169823
Most similar script: [SEARCH ONE FROM:SUPERMARKET WHERE:ONROUTE WITH:CHARGINGSTATION]
Estimation: 6
Answer: 6

